# Playground

## Import libraries

In [22]:
import getpass
import os
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup, Comment

In [16]:
load_dotenv()

True

## Test api connection

In [17]:
if not os.environ.get("TOGETHER_API_KEY"):
    os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter API key for Together AI: ")

In [28]:
# meta-llama/Llama-3.3-70B-Instruct-Turbo-Free   inputs` tokens + `max_new_tokens` must be <= 8193
# Qwen/Qwen2.5-Coder-32B-Instruct                inputs` tokens + `max_new_tokens` must be <= 32769
model = init_chat_model("meta-llama/Llama-3.2-3B-Instruct-Turbo", model_provider="together")

In [19]:
messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!")
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 22, 'total_tokens': 26, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen2.5-Coder-32B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9802982-fda8-4565-8a25-bf4dfc5fedf9-0', usage_metadata={'input_tokens': 22, 'output_tokens': 4, 'total_tokens': 26, 'input_token_details': {}, 'output_token_details': {}})

## Python HTML request

In [ ]:
repertoire_page = "https://teatrdramatyczny.pl/whats-on"
response = requests.get(repertoire_page)

if response.status_code == 200:
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()
    body_content = soup.body
    with open("repertoire_body.html", "w", encoding="utf-8") as file:
        file.write(str(body_content))
    print("HTML content saved to 'repertoire.html'")
else:
    print(f"Failed to retrieve content: {response.status_code}")


HTML content saved to 'repertoire.html'


## Parse HTML with LLM

In [29]:
with open("repertoire_body.html", "r", encoding="utf-8") as file:
    html_content = file.read()
system_message = """
I will provide you with an HTML snippet containing information about theater performances. Extract all performances, including their titles, dates, and times, and return the result as a JSON array with the following format:
```
[
  {
    "title": "Performance Name",
    "date": "YYYY-MM-DD",
    "time": "HH:MM"
  }
]
```
Do not include any extra text in the response—only return valid JSON.
"""
messages = [
    SystemMessage(system_message),
    HumanMessage(html_content)
]

response = model.invoke(messages)

In [31]:
with open("llm_response.txt", "w") as file:
    file.write(response.content)